In [2]:
import pandas as pd

In [50]:
raw_affective = pd.read_csv("/Users/calebjonesshibu/Desktop/tom/data/exp_2023_02_03_10/baseline_tasks/affective/individual_00104_1675449225.csv", sep=";")
extracted_affective = pd.read_csv("/Users/calebjonesshibu/Desktop/Neurips/Affective_Task_Individual/exp_2023_02_03_10/104.csv", sep=",")
# extracted_affective.drop("Unnamed: 0", axis=1, inplace=True)

In [53]:
# raw_affective['arousal_score'].fillna(method='ffill', inplace=True)
# raw_affective['valence_score'].fillna(method='ffill', inplace=True)
# raw_affective['subject_id'].fillna(method='ffill', inplace=True)
# Forward fill NaN values in all columns
raw_affective.fillna(method='bfill', inplace=True)
extracted_affective.fillna(method='bfill', inplace=True)

In [54]:
raw_affective

,time,monotonic_time,human_readable_time,image_path,subject_id,arousal_score,valence_score,event_type
0,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
1,1.675449e+09,244205.959496,2023-02-03T18:33:55.742628Z,individual000.jpg,104.0,1.0,0.0,intermediate_selection
2,1.675449e+09,244207.328055,2023-02-03T18:33:57.111187Z,individual000.jpg,104.0,1.0,0.0,intermediate_selection
3,1.675449e+09,244208.145096,2023-02-03T18:33:57.928230Z,individual000.jpg,104.0,1.0,0.0,final_submission
4,1.675449e+09,244215.823509,2023-02-03T18:34:05.606641Z,individual001.jpg,104.0,0.0,0.0,intermediate_selection
5,1.675449e+09,244217.633949,2023-02-03T18:34:07.417081Z,individual001.jpg,104.0,0.0,0.0,intermediate_selection
6,1.675449e+09,244219.105550,2023-02-03T18:34:08.888684Z,individual001.jpg,104.0,0.0,0.0,final_submission
7,1.675449e+09,244228.566381,2023-02-03T18:34:18.349513Z,individual002.jpg,104.0,0.0,0.0,intermediate_selection
8,1.675449e+09,244229.655970,2023-02-03T18:34:19.439102Z,individual002.jpg,104.0,0.0,0.0,intermediate_selection
9,1.675449e+09,244230.961826,2023-02-03T18:34:20.744960Z,individual002.jpg,104.0,0.0,0.0,final_submission


In [57]:
extracted_affective

,S1-D1_HbO,S1-D2_HbO,S2-D1_HbO,S2-D3_HbO,S3-D1_HbO,S3-D3_HbO,S3-D4_HbO,S4-D2_HbO,S4-D4_HbO,S4-D5_HbO,...,unix_time,event_type_x,time,monotonic_time,human_readable_time_y,image_path,subject_id,arousal_score,valence_score,event_type_y
0,0.245596,0.504800,0.643060,1.078668,0.435682,0.805907,0.115973,-0.023459,0.111901,0.207936,...,1.675449e+09,affective_task_individual,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
1,0.259480,0.529802,0.678952,1.141680,0.225865,0.850175,0.125032,-0.010211,0.125048,0.229969,...,1.675449e+09,affective_task_individual,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
2,0.274110,0.550814,0.715220,1.204581,0.018815,0.895324,0.135430,0.005934,0.138927,0.251522,...,1.675449e+09,affective_task_individual,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
3,0.289508,0.567740,0.751839,1.267116,-0.178001,0.941189,0.147121,0.025163,0.153410,0.272510,...,1.675449e+09,affective_task_individual,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
4,0.305641,0.580463,0.788612,1.328906,-0.355855,0.987512,0.159818,0.047384,0.168465,0.292811,...,1.675449e+09,affective_task_individual,1.675449e+09,244195.457899,2023-02-03T18:33:45.241038Z,individual000.jpg,104.0,1.0,0.0,start_affective_task
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,0.708500,0.314289,0.535449,0.635889,-0.931893,0.986629,0.990487,0.996710,1.027527,1.474123,...,1.675449e+09,affective_task_individual,1.675449e+09,244372.653689,2023-02-03T18:36:42.436821Z,individual014.jpg,104.0,0.0,-1.0,intermediate_selection
1804,0.709637,0.326118,0.535126,0.625569,-0.831574,0.986360,1.000301,0.991931,1.029388,1.455650,...,1.675449e+09,affective_task_individual,1.675449e+09,244372.653689,2023-02-03T18:36:42.436821Z,individual014.jpg,104.0,0.0,-1.0,intermediate_selection
1805,0.711477,0.337099,0.536006,0.616891,-0.715133,0.986748,1.010629,0.986372,1.031153,1.437100,...,1.675449e+09,affective_task_individual,1.675449e+09,244373.231040,2023-02-03T18:36:43.014173Z,individual014.jpg,104.0,0.0,-1.0,final_submission
1806,0.714147,0.347088,0.538153,0.609951,-0.590154,0.987926,1.021720,0.980145,1.033003,1.418573,...,1.675449e+09,affective_task_individual,1.675449e+09,244373.231040,2023-02-03T18:36:43.014173Z,individual014.jpg,104.0,0.0,-1.0,final_submission


In [56]:
import pandas as pd

# Assuming you have the DataFrames raw_affective and extracted_affective

# Convert 'human_readable_time' column in extracted_affective to datetime type
extracted_affective['human_readable_time'] = pd.to_datetime(extracted_affective['human_readable_time'])

# Perform the merge based on closest matching time
merged_df = pd.merge_asof(extracted_affective.sort_values('unix_time'),
                          raw_affective.sort_values('time'),
                          left_on='unix_time',
                          right_on='time',
                          direction='nearest')

KeyError: 'human_readable_time'

In [46]:
merged_df.to_csv("/Users/calebjonesshibu/Desktop/101.csv", index=False)

In [48]:
raw_affective.sort_values('time')

,time,monotonic_time,human_readable_time,image_path,subject_id,arousal_score,valence_score,event_type
0,1.675449e+09,244195.416228,2023-02-03 18:33:45.199366+00:00,individual000.jpg,101.0,0.0,0.0,start_affective_task
1,1.675449e+09,244205.515779,2023-02-03 18:33:55.298911+00:00,individual000.jpg,101.0,0.0,0.0,intermediate_selection
2,1.675449e+09,244206.581171,2023-02-03 18:33:56.364303+00:00,individual000.jpg,101.0,0.0,0.0,intermediate_selection
3,1.675449e+09,244207.541960,2023-02-03 18:33:57.325092+00:00,individual000.jpg,101.0,0.0,0.0,final_submission
4,1.675449e+09,244217.197414,2023-02-03 18:34:06.980546+00:00,individual001.jpg,101.0,1.0,-1.0,intermediate_selection
5,1.675449e+09,244219.093744,2023-02-03 18:34:08.876876+00:00,individual001.jpg,101.0,1.0,-1.0,intermediate_selection
6,1.675449e+09,244220.182191,2023-02-03 18:34:09.965324+00:00,individual001.jpg,101.0,1.0,-1.0,final_submission
7,1.675449e+09,244231.270463,2023-02-03 18:34:21.053594+00:00,individual002.jpg,101.0,1.0,1.0,intermediate_selection
8,1.675449e+09,244232.886761,2023-02-03 18:34:22.669893+00:00,individual002.jpg,101.0,1.0,1.0,intermediate_selection
9,1.675449e+09,244233.438639,2023-02-03 18:34:23.221772+00:00,individual002.jpg,101.0,1.0,1.0,final_submission
